Only reading in one email for now

In [11]:
emailDir = "../maildir/maildir"
enroncsv = "../experiments/enron.csv"

Using the built in python email parser to get all the info required

In [15]:
columns = ['Filename',
            'Directory',
            'Message-ID',
            'Date',
            'From',
            'To',
            'Subject',
            'Cc',
            'Time',
            'Attendees',
            'Re',
            'Mime-Version',
            'Content-Type',
            'Content-Transfer-Encoding',
            'Bcc',
            'X-From',
            'X-To',
            'X-cc',
            'X-bcc',
            'X-Folder',
            'X-Origin',
            'X-FileName']
columns

['Filename',
 'Directory',
 'Message-ID',
 'Date',
 'From',
 'To',
 'Subject',
 'Cc',
 'Mime-Version',
 'Content-Type',
 'Content-Transfer-Encoding',
 'Bcc',
 'X-From',
 'X-To',
 'X-cc',
 'X-bcc',
 'X-Folder',
 'X-Origin',
 'X-FileName']

In [ ]:
import os
import csv

from email.parser import BytesParser, Parser
from email.policy import default

i = 0
with open(enroncsv, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, delimiter='|', quotechar='|', fieldnames=columns)
    for root, dirs, files in os.walk(emailDir):
        for name in files:
            i += 1
            if i % 1000 == 0:
                print(i)
            with open(os.path.join(root, name), 'rb') as fp:
                email = BytesParser(policy=default).parse(fp)
                email['Filename'] = name
                try:
                    writer.writerow(email)
                except Exception as e:
                    print("Error: Couldn't write email")
                    print(e)
                    

1000
2000


Using the `|` seperator here so that we don't have issues with `,` in the data. Don't 100% there are no pipes but it has worked so far.

In [ ]:
df = pd.read_csv(enroncsv, names=columns, sep='|')

result = df.loc[df['To'] == 'pallen@enron.com']
result